# Plotting_SANS_Data

This script will create SANS/USANS data plots in conjugated polymer concentration series. The USANS data will be plotting in the smeared form.

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt

from bumps.names import *
from bumps.fitters import fit

import sasmodels

from sasmodels.core import load_model
from sasmodels.bumps_model import Model, Experiment
from sasmodels.data import load_data, plot_data, empty_data1D
from sasmodels.direct_model import DirectModel

import sas

### Loading Sample Data and Meta-Data

In [3]:
# sample meta-data
sample_info = pd.read_csv('../data/sans/Sample_Info.csv')

# helpful meta-data dictionaries
names = dict(zip(sample_info['Sample'], sample_info['Name']))
cps = dict(zip(sample_info['Sample'], sample_info['Conjugated Polymer']))
matrix = dict(zip(sample_info['Sample'], sample_info['Commodity Polymer']))
solvent_names = dict(zip(sample_info['Sample'], sample_info['Solvent']))

# target weight percents of conjugated polymer
target = dict(zip(sample_info['Sample'], sample_info['Target Fraction']*100))

# fixing 401/402 and 403/404 sample pair target values for plotting colors only
target[401] = 0.5
target[402] = 0.1
target[403] = 5
target[404] = 1

# actual weight percentages
data = np.loadtxt('../data/uv_vis/Corrected_wtPercents.csv', delimiter=',', skiprows=1)
actual = {}
for key, tar, act, stdev, act_vol, stdev_vol in data:
    actual[key] = act


Loading reduced and compiled SANS and USANS data.

In [4]:
data_dir = '../data/sans/Smeared_Data_20200629/'
files = os.listdir(data_dir)
sans_data = {}
usans_data = {}
for file in files:
    if 'USANS' in file:
        key = int(file.split('_')[0][3:])
        usans_data[key] = load_data(data_dir + file)
    elif 'SANS' in file:
        key = int(file.split('_')[0][3:])
        sans_data[key] = load_data(data_dir + file)

In [5]:
# useful dictionaries with labels and colors for the plots and their legends
wt_names = {} 
full_names = {}

wt_colors = {}
solvent_colors = {}
cp_colors = {}

rep_colors = {}
rep_names = {}

temp_wt_colors = {
    0.1: 'firebrick',
    0.5: 'darkorange',
    1.0: 'darkcyan',
    5.0: 'mediumblue',
    10.0: 'deeppink',
    25.0: 'darkorchid',
    50.0: 'forestgreen',
    0.0: 'black'
}

temp_solvent_colors = {
    'Chloroform': 'firebrick',
    'Bromobenzene': 'darkorange',
    'Toluene': 'darkcyan',
    'Slow Dry Chloroform': 'darkorchid'
}

temp_cp_colors = {
    'RRe-P3HT': 'firebrick',
    'RRa-P3HT': 'darkorange',
    'P3DDT': 'darkorchid',
    'PQT-12': 'darkcyan',
    'None': 'black'
}

for key in names.keys():
    if key in actual.keys():
        frac = actual[key]
    else:
        frac = target[key]
    frac = np.round(frac,2)
    
    if cps[key] == 'None':
        wt_names[key] = matrix[key] + ' Control'
        full_names[key] = matrix[key] + ' Control'
    else:
        wt_names[key] = str(frac) + ' wt% ' + cps[key]
        full_names[key] = str(frac) + ' wt% ' + cps[key] + ' in ' + matrix[key]
        
for key in cps.keys():
    wt_colors[key] = temp_wt_colors[target[key]]
    solvent_colors[key] = temp_solvent_colors[solvent_names[key]]
    cp_colors[key] = temp_cp_colors[cps[key]]


### Creating common plotting function

In [9]:
def plot_sans(keys,sans_data_dict,usans_data_dict,zorders,label_dict,color_dict,filename,title=None,legend_title=None, size=5):
    
    make_colors = {}
    if type(color_dict) is list:
        print('converting color list')
        i = 0
        for key in keys:
            make_colors[key] = color_dict[i]
            i += 1
        color_dict = make_colors
    
    make_labels = {}
    if type(label_dict) is list:
        print('converting label list')
        i = 0
        for key in keys:
            make_labels[key] = label_dict[i]
            i += 1
        label_dict = make_labels
            
            
    plt.figure(figsize=(6,6))
    i = 0
    for key in keys:
        sans_data = sans_data_dict[key]
        
        plt.errorbar(sans_data.x, sans_data.y, yerr=sans_data.dy, zorder = zorders[i], label=label_dict[key], c=color_dict[key], ms=size, fmt='o')
        
        if key in usans_data_dict.keys():
            usans_data = usans_data_dict[key]
            plt.errorbar(usans_data.x, usans_data.y, yerr=usans_data.dy, zorder = zorders[i], label=None, c=color_dict[key], ms=size, fmt='o')
                
    plt.xscale('log')
    plt.yscale('log')

    plt.xlabel(r'Q ($\AA^{-1}$)', fontsize=16)
    plt.ylabel(r'I(Q) (cm$^{-1}$)', fontsize=16)
    
    legend = plt.legend(fontsize=12, title=legend_title)
    legend.get_title().set_fontsize('12')

    plt.xticks(fontsize=14)
    plt.yticks(fontsize=14)

    plt.ylim(1e-03, 1e11)
    plt.xlim(0.00002, 0.3)

    if title is not None:
        plt.title(title, fontsize=16)
    
    plt.tight_layout()

    plt.savefig(filename, dpi=600)
    
    plt.close()

### Plotting Concentration Series

In [12]:
save_loc = '../data/sans/SANS_Figures/'
os.makedirs(save_loc, exist_ok=True)

plot_sans([1,2,3,4,5,21],sans_data,usans_data,[5,4,3,2,1,6],wt_names,wt_colors, save_loc + 'RReP3HT_CF_1.png',title='RRe-P3HT in PS-D8 from Chloroform')
plot_sans([6,7,8,9,10,21],sans_data,usans_data,[5,4,3,2,1,6],wt_names,wt_colors, save_loc +'RRaP3HT_CF_1.png',title='RRa-P3HT in PS-D8 from Chloroform')
plot_sans([11,12,13,14,15,21],sans_data,usans_data,[5,4,3,2,1,6],wt_names,wt_colors, save_loc +'P3DDT_CF.png',title='P3DDT in PS-D8 from Chloroform')
plot_sans([16,17,18,19,20,21],sans_data,usans_data,[5,4,3,2,1,6],wt_names,wt_colors, save_loc +'PQT12_CF.png',title='PQT-12 in PS-D8 from Chloroform')

plot_sans([101,102,103,104,105,106,107,113],sans_data,usans_data,[7,6,5,4,3,2,1,8],wt_names,wt_colors, save_loc +'RReP3HT_CF_2.png',title='RRe-P3HT in PS-D8 from Chloroform')
plot_sans([108,109,110,111,112,114],sans_data,usans_data,[5,4,3,2,1,6],wt_names,wt_colors, save_loc +'RReP3HT_CF_3.png',title='RRe-P3HT in PS-D8 from Chloroform')


plot_sans([22,23,24,28],sans_data,usans_data,[3,2,1,4],wt_names,wt_colors, save_loc +'RReP3HT_BB_1.png',title='RRe-P3HT in PS-D8 from Bromobenzene')
plot_sans([25,26,27,28],sans_data,usans_data,[3,2,1,4],wt_names,wt_colors, save_loc +'P3DDT_BB.png',title='P3DDT in PS-D8 from Bromobenzene')

plot_sans([29,30,31,35],sans_data,usans_data,[3,2,1,4],wt_names,wt_colors, save_loc +'RReP3HT_Tol_1.png',title='RRe-P3HT in PS-D8 from Toluene')
plot_sans([32,33,34,35],sans_data,usans_data,[3,2,1,4],wt_names,wt_colors, save_loc +'P3DDT_Tol.png',title='P3DDT in PS-D8 from Toluene')

plot_sans([40,41,42,43],sans_data,usans_data,[3,2,1,4],wt_names,wt_colors, save_loc +'RReP3HT_CF_PSH8.png',title='RRe-P3HT in PS-H8 from Chloroform')

plot_sans([203,113],sans_data,usans_data,[1,2],wt_names,wt_colors, save_loc +'RRaP3HT_CF_2.png',title='RRa-P3HT in PS-D8 from Chloroform')
plot_sans([205,206,114],sans_data,usans_data,[2,1,3],wt_names,wt_colors, save_loc +'RRaP3HT_CF_3.png',title='RRa-P3HT in PS-D8 from Chloroform')

plot_sans([301,302,303,304,305,310],sans_data,usans_data,[5,4,3,2,1,6],wt_names,wt_colors, save_loc +'RReP3HT_BB_2.png',title='RRe-P3HT in PS-D8 from Bromobenzene')
plot_sans([306,307,309],sans_data,usans_data,[2,1,3],wt_names,wt_colors, save_loc +'RReP3HT_BB_3.png',title='RRe-P3HT in PS-D8 from Bromobenzene')

plot_sans([402,401,404,403,405,409],sans_data,usans_data,[5,4,3,2,1,6],wt_names,wt_colors, save_loc +'RReP3HT_Tol_2.png',title='RRe-P3HT in PS-D8 from Toluene')
plot_sans([406,407,408,409],sans_data,usans_data,[3,2,1,4],wt_names,wt_colors, save_loc +'RReP3HT_Tol_3.png',title='RRe-P3HT in PS-D8 from Toluene')

plot_sans([501,502,503,504,505,506],sans_data,usans_data,[5,4,3,2,1,6],wt_names,wt_colors, save_loc +'RReP3HT_CF_SLOW.png',title='RRe-P3HT in PS-D8 from Chloroform - Slow Dry')